# Modelling test

### Importing the libraries

In [65]:
import pandas as pd
import os
import json
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Preparing the data

In [66]:
os.chdir('C:\\Users\\hp\\Downloads\\senior_ds_test\\data\\train')

In [67]:
with open('accounts_data_train.json') as f:
   data = json.load(f)

flattened_data = [item for sublist in data for item in sublist]
accounts = pd.DataFrame(flattened_data)

In [68]:
with open('enquiry_data_train.json') as f:
   data = json.load(f)

flattened_data = [item for sublist in data for item in sublist]
enquiry = pd.DataFrame(flattened_data)

As the data consisted of a list of dictionaries it wasn't possible to import it directly using pandas.read_json. Here I have flattened the data first and then converted it to a dataframe

In [69]:
flag = pd.read_csv("train_flag.csv")

In [70]:
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245310 entries, 0 to 1245309
Data columns (total 7 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   credit_type          1245310 non-null  object 
 1   loan_amount          1245307 non-null  float64
 2   amount_overdue       1245310 non-null  float64
 3   open_date            1245310 non-null  object 
 4   closed_date          782275 non-null   object 
 5   payment_hist_string  1245310 non-null  object 
 6   uid                  1245310 non-null  object 
dtypes: float64(2), object(5)
memory usage: 66.5+ MB


In [71]:
enquiry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1909926 entries, 0 to 1909925
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   enquiry_type  object
 1   enquiry_amt   int64 
 2   enquiry_date  object
 3   uid           object
dtypes: int64(1), object(3)
memory usage: 58.3+ MB


In [72]:
flag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261383 entries, 0 to 261382
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   uid                 261383 non-null  object
 1   NAME_CONTRACT_TYPE  261383 non-null  object
 2   TARGET              261383 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 6.0+ MB


As we would need to merge the datasets to create a dataframe for our model we need to check the rows which are present in all the data frames

In [73]:
accounts[accounts['uid'].isin(enquiry['uid']) & accounts['uid'].isin(flag['uid'])]['uid'].nunique()

223918

This shows that we have 223918 entries common for all the datasets. Finally we will merge the datasets to create our final data for modelling

In [74]:
data=pd.merge(pd.merge(accounts,enquiry,on='uid'),flag,on='uid')

In [75]:
data.drop_duplicates(subset=['uid'],inplace=True)

### EDA of the final data

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223918 entries, 0 to 9102571
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   credit_type          223918 non-null  object 
 1   loan_amount          223916 non-null  float64
 2   amount_overdue       223918 non-null  float64
 3   open_date            223918 non-null  object 
 4   closed_date          151833 non-null  object 
 5   payment_hist_string  223918 non-null  object 
 6   uid                  223918 non-null  object 
 7   enquiry_type         223918 non-null  object 
 8   enquiry_amt          223918 non-null  int64  
 9   enquiry_date         223918 non-null  object 
 10  NAME_CONTRACT_TYPE   223918 non-null  object 
 11  TARGET               223918 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 22.2+ MB


In [77]:
data.isnull().sum()

credit_type                0
loan_amount                2
amount_overdue             0
open_date                  0
closed_date            72085
payment_hist_string        0
uid                        0
enquiry_type               0
enquiry_amt                0
enquiry_date               0
NAME_CONTRACT_TYPE         0
TARGET                     0
dtype: int64

We already know that the closed_date has null data. We will transform it and remove the rows with missing loan amount.

In [78]:
data['closed_date'] = data['closed_date'].fillna(0)

In [79]:
data = data.dropna()

After dropping all the NA values, we need to factorize the categorical variables so that we can use the data to fit our model.

In [80]:
data[['credit_type', 'enquiry_type', 'NAME_CONTRACT_TYPE']] = data[['credit_type', 'enquiry_type', 'NAME_CONTRACT_TYPE']].apply(lambda x: pd.factorize(x)[0])

Finally, we convert the payment_hist_string to a variable days_overdue to get an idea of how many days the borrower was overdue by

In [81]:
def days_overdue(list):
    a = sum(int(month) for month in list)
    return a

In [82]:
data['days_overdue'] = data['payment_hist_string'].apply(days_overdue)

In [83]:
data['loan_cleared'] = data['closed_date'].apply(lambda x: 0 if x == 0 else 1)

Next we will use the closed_date variable to check if the borrower cleared the previous loan or not.

In [84]:
corr_matrix = data.corr()
corr_matrix

,credit_type,loan_amount,amount_overdue,enquiry_type,enquiry_amt,NAME_CONTRACT_TYPE,TARGET,days_overdue,loan_cleared
credit_type,1.000000,0.258159,0.009856,-0.001814,0.016379,0.023885,0.009319,0.023987,-0.364794
loan_amount,0.258159,1.000000,0.002678,0.000551,0.008902,0.014513,-0.013363,0.012837,-0.094128
amount_overdue,0.009856,0.002678,1.000000,-0.000638,-0.000524,0.002437,0.006518,0.056521,-0.009079
enquiry_type,-0.001814,0.000551,-0.000638,1.000000,0.003292,0.007376,-0.002288,0.000072,0.000283
enquiry_amt,0.016379,0.008902,-0.000524,0.003292,1.000000,0.527877,-0.015379,0.001045,-0.010210
NAME_CONTRACT_TYPE,0.023885,0.014513,0.002437,0.007376,0.527877,1.000000,-0.030450,-0.001038,-0.017117
TARGET,0.009319,-0.013363,0.006518,-0.002288,-0.015379,-0.030450,1.000000,0.002071,-0.042780
days_overdue,0.023987,0.012837,0.056521,0.000072,0.001045,-0.001038,0.002071,1.000000,-0.049653
loan_cleared,-0.364794,-0.094128,-0.009079,0.000283,-0.010210,-0.017117,-0.042780,-0.049653,1.000000


Finally we check the correlation matrix to check how the variables are correlated to each other. Also we will check how they are related to the Target variable that we need to predict

In [85]:
corr_matrix['TARGET'].abs().sort_values(ascending=False)

TARGET                1.000000
loan_cleared          0.042780
NAME_CONTRACT_TYPE    0.030450
enquiry_amt           0.015379
loan_amount           0.013363
credit_type           0.009319
amount_overdue        0.006518
enquiry_type          0.002288
days_overdue          0.002071
Name: TARGET, dtype: float64

In [86]:
X = data.drop(['open_date','closed_date', 'uid','enquiry_date','TARGET','payment_hist_string'], axis = 1)

In [87]:
Y = data['TARGET']

In [88]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

As we don't have the test outputs, we will use the train set to create the validation set to check how well our models can perform on unseen data

### Creating the models

For the purpose of the problem, we will create different models using both regression and classification to check which model provides the best auc score for the data

In [89]:
model = LogisticRegression()
model.fit(x_train, y_train)
auc = roc_auc_score(y_val, model.predict(x_val))
print("AUC Score:", auc)

AUC Score: 0.5


In [90]:
tree = DecisionTreeRegressor(max_depth=5, random_state=42)
tree.fit(x_train, y_train)
auc1 = roc_auc_score(y_val, tree.predict(x_val))
print("AUC Score:", auc1)

AUC Score: 0.5627958287954888


In [91]:
rf = RandomForestRegressor(n_estimators = 10)
rf.fit(x_train, y_train)
auc2 = roc_auc_score(y_val, rf.predict(x_val))
print("AUC Score:", auc2)

AUC Score: 0.508727014024075


In [93]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(x_train, y_train)
auc3 = roc_auc_score(y_val, rfc.predict(x_val))
print("AUC Score:", auc3)

AUC Score: 0.4988931623630984


From the training data we see that Decision Tree Regressor provides the best result for the data. Although, it is not a great model but does give better predictions than the others.

In [101]:
print(data[data['TARGET']==0]['TARGET'].count())
print(data[data['TARGET']==1]['TARGET'].count())

206659
17257


From the count of TARGET variable which we are predicting, it can be seen that there is a severe imbalance between the two classes which may be the cause behind the low auc score for the models here.

### Hyperparameter tuning

For our chosen model, we will check different parameters to provide most optimal results. 

In [102]:
param_grid = {
    'max_depth': [5, 10, 20]
}

In [103]:
dt = DecisionTreeRegressor()

In [104]:
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X,Y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 20]}, verbose=2)

In [105]:
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
best_params

{'max_depth': 5}

In [106]:
best_estimator.fit(X, Y)

DecisionTreeRegressor(max_depth=5)

In [107]:
auc = roc_auc_score(y_val, best_estimator.predict(x_val))
print("AUC Score:", auc)

AUC Score: 0.5661447363851545


### Testing on test data

In [108]:
os.chdir('C:\\Users\\hp\\Downloads\\senior_ds_test\\data\\test')

We will need to preprocess the test data just like we did with the training data to provide the final predictions

In [109]:
with open('accounts_data_test.json') as f:
   data = json.load(f)

flattened_data = [item for sublist in data for item in sublist]
accounts_test = pd.DataFrame(flattened_data)

In [110]:
with open('enquiry_data_test.json') as f:
   data = json.load(f)

flattened_data = [item for sublist in data for item in sublist]
enquiry_test = pd.DataFrame(flattened_data)

In [111]:
flag_test = pd.read_csv("test_flag.csv")

In [112]:
data_test=pd.merge(pd.merge(accounts_test,enquiry_test,on='uid'),flag_test,on='uid')

In [113]:
data_test.drop_duplicates(subset=['uid'],inplace=True)

In [114]:
data_test.isnull().sum()

credit_type                0
loan_amount                0
amount_overdue             0
open_date                  0
closed_date            12876
payment_hist_string        0
uid                        0
enquiry_type               0
enquiry_amt                0
enquiry_date               0
NAME_CONTRACT_TYPE         0
dtype: int64

In [115]:
data_test['closed_date'] = data_test['closed_date'].fillna(0)

In [116]:
data_test[['credit_type', 'enquiry_type', 'NAME_CONTRACT_TYPE']] = data_test[['credit_type', 'enquiry_type', 'NAME_CONTRACT_TYPE']].apply(lambda x: pd.factorize(x)[0])

In [117]:
data_test['days_overdue'] = data_test['payment_hist_string'].apply(days_overdue)

In [118]:
data_test['loan_cleared'] = data_test['closed_date'].apply(lambda x: 0 if x == 0 else 1)

In [119]:
x_test = data_test.drop(['open_date','closed_date', 'uid','enquiry_date','payment_hist_string'], axis = 1)

In [120]:
y_test=best_estimator.predict(x_test)

We will use the best_estimator which we got after hyperparameter tuning to provide the predictions.

In [121]:
final = pd.DataFrame({'uid': data_test['uid'], 'pred': y_test})

In [122]:
final.to_csv('file_submission_Siddhesh_Bagwe.csv')